In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

# Cloud type index = 37
# clear = 0
# stratus continental = 1
# stratus maritime = 2
# cumulus continental clean = 3
# cumulus continental polluted = 4
# cumulus maritime = 5
# cirrus = 6
# cirrus + water clouds (mixed) >= 7

USE_MIXED = True
USE_PRESSURE_TEMP = True
USE_SPECIFIC_CLOUDS = True
LOW_VIS_FRAC = True
low_vis_frac = 0.4
low_vis_threshold = 25000 #50000
cloud_types = [6] #0,1,2 ::: 0, 3, 4
noise_level = 0.01 # % of max
dnn_batch_size = 64

if USE_MIXED:
    filenames_vis = ['fog_data2_vis_clear.dat', 'fog_data2_vis_ice.dat', 'fog_data2_vis_mixed.dat', 'fog_data2_vis_water.dat']
    filenames_ir = ['fog_data2_ir_clear.dat', 'fog_data2_ir_ice.dat', 'fog_data2_ir_mixed.dat', 'fog_data2_ir_water.dat']
else:
    filenames_vis = ['fog_data2_vis_clear.dat', 'fog_data2_vis_ice.dat', 'fog_data2_vis_water.dat']
    filenames_ir = ['fog_data2_ir_clear.dat', 'fog_data2_ir_ice.dat', 'fog_data2_ir_water.dat']

target_i = 30 # Visibility index
if USE_PRESSURE_TEMP:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,36))) # 13 ir bands, 10 vis bands, 3 angles, pressure, temp
else:
    features_i = np.concatenate((np.arange(4, 17), np.arange(20, 30), np.arange(31,34)))

# Combine .dat files into one numpy array
data_ir_vis = []
for file_ir, file_vis in zip(filenames_ir, filenames_vis):
  raw_data_ir = pd.read_csv('fog_dataset2/' + file_ir).to_numpy()
  raw_data_vis = pd.read_csv('fog_dataset2/' + file_vis).to_numpy()

  data_ir = np.array([row[0].split() for row in raw_data_ir[25:]])
  data_vis = np.array([row[0].split()[1:] for row in raw_data_vis[41:]])

  if len(data_ir_vis) == 0:
    data_ir_vis = np.hstack([data_ir, data_vis])
  else:
    data_ir_vis = np.vstack([data_ir_vis, np.hstack([data_ir, data_vis])])

# Remove surface description to convert to float
data_ir_vis = data_ir_vis[:,:-1] 
data_ir_vis = data_ir_vis.astype(np.float32)

# 19 Data points contains nan. Not sure why. Band 32 culprit, 8th feature
nan_i = np.where(np.isnan(data_ir_vis))[0]
data_ir_vis = np.delete(data_ir_vis, nan_i, axis=0)

# Cloud type index = 37
if USE_SPECIFIC_CLOUDS:
   data_ir_vis = data_ir_vis[np.isin(data_ir_vis[:, 37], cloud_types)]


X = data_ir_vis[:, features_i]
y = data_ir_vis[:,target_i]

noise_std_devs = np.zeros(features_i.shape)

for i, feature_i in enumerate(features_i):
    noise_std = (np.max(data_ir_vis[:,feature_i]) - np.min(data_ir_vis[:,feature_i])) * noise_level # 1% of mean or max??
    noise_std_devs[i] = noise_std

# Scaling features and noise standard deviations
feature_scaler = StandardScaler()
X_scaled = feature_scaler.fit_transform(X)
noise_stds_scaled = np.divide(noise_std_devs, feature_scaler.scale_)

# Scaling target
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(y.reshape(-1, 1))


# First split: Separate out a test set (5% of the original dataset)
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y_scaled, test_size=(1/20))


if LOW_VIS_FRAC:
    low_vis_i = np.where(target_scaler.inverse_transform(y_temp) < low_vis_threshold)[0]
    high_vis_i = np.where(target_scaler.inverse_transform(y_temp) >= low_vis_threshold)[0]

    n_high_vis = int(len(low_vis_i) / low_vis_frac * (1 - low_vis_frac))
    print(f'real frac: {len(low_vis_i)/len(y_scaled)}')
    if n_high_vis > len(high_vis_i):
        n_high_vis = len(high_vis_i)
        n_low_vis = int(low_vis_frac * n_high_vis)
        r_low_vis_i = np.random.choice(low_vis_i, n_low_vis, replace=False)
        low_vis_i = r_low_vis_i
    print(f'n_low_vis: {len(low_vis_i)}. n_high_vis: {n_high_vis}')
    r_high_vis_i = np.random.choice(high_vis_i, n_high_vis, replace=False)
    X_temp = np.vstack([X_temp[low_vis_i], X_temp[r_high_vis_i]])
    y_temp = np.vstack([y_temp[low_vis_i], y_temp[r_high_vis_i]])

  

# Second split: Split the remaining data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=(1/19))

# Create noisy data
X_train_noisy = X_train + np.multiply(np.random.normal(0, 1, X_train.shape), noise_stds_scaled)
X_val_noisy = X_val + np.multiply(np.random.normal(0, 1, X_val.shape), noise_stds_scaled)
X_test_noisy = X_test + np.multiply(np.random.normal(0, 1, X_test.shape), noise_stds_scaled)

# Create class labels
y_train_label = np.where(target_scaler.inverse_transform(y_train) < low_vis_threshold, 1, 0)
y_val_label = np.where(target_scaler.inverse_transform(y_val) < low_vis_threshold, 1, 0)
y_test_label = np.where(target_scaler.inverse_transform(y_test) < low_vis_threshold, 1, 0)


# Create training data for DNN that adds noise to batches
training_data = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size=dnn_batch_size)
training_data_classes = tf.data.Dataset.from_tensor_slices((X_train, y_train_label)).batch(batch_size=dnn_batch_size)

def noise_map(features, labels):
    noise = tf.random.normal(shape=tf.shape(features))
    noise = noise * noise_stds_scaled
    noisy_features = features + noise
    return noisy_features, labels

training_data = training_data.map(noise_map)
training_data_classes = training_data_classes.map(noise_map)


2024-03-13 15:32:47.540059: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 15:32:47.574932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 15:32:47.574953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 15:32:47.575841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 15:32:47.581861: I tensorflow/core/platform/cpu_feature_guar

real frac: 0.15480429419217176
n_low_vis: 23216. n_high_vis: 34824


2024-03-13 15:33:14.319214: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 15:33:14.354769: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 15:33:14.357014: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
from qrnn import get_model, qloss
from keras.callbacks import *
import numpy as np


# Parameters
input_dim = len(features_i)
num_hidden_layers = 4
num_units = [64, 64, 64, 64]
act = ['relu', 'relu', 'relu', 'relu']
dropout = [0.1, 0.1, 0.1, 0.1]
gauss_std = [0.0, 0.0, 0.0, 0.0]
num_quantiles = 9

# Get model
model = get_model(input_dim, num_units, act, dropout, gauss_std, num_hidden_layers, num_quantiles)
print(model.summary())

# Train
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss=lambda y_t, y_p: qloss(y_true=y_t, y_pred=y_p, n_q=num_quantiles), 
              optimizer='adam')
model.fit(x=X_train_noisy, y=y_train, 
          epochs=40, 
          validation_split=0.1, 
          batch_size=64, 
          shuffle=True, 
          callbacks=[early_stopping]
         )

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         1,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_1                │ (None, 64)             │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_2                │ (None, 64)             │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_3                │ (None, 64)             │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,921 (58.29 KB)

 Trainable params: 14,921 (58.29 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/40


2024-03-13 15:33:32.767880: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f424c002690 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 15:33:32.767914: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-13 15:33:32.798924: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-13 15:33:33.089186: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


 78/774 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.0995

I0000 00:00:1710344016.974336 1253972 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


774/774 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 0.7099 - val_loss: 0.3126
Epoch 2/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3083 - val_loss: 0.2295
Epoch 3/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2412 - val_loss: 0.2069
Epoch 4/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2206 - val_loss: 0.1962
Epoch 5/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2133 - val_loss: 0.1914
Epoch 6/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2093 - val_loss: 0.1891
Epoch 7/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2053 - val_loss: 0.1858
Epoch 8/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2032 - val_loss: 0.1851
Epoch 9/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2040 - val_loss: 0.1849
Epoch 10/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2006 - val_loss: 0.1842
Epoch 11/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.2000 - val_loss: 0.1813
Epoch 12/40
774/774 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0

In [1]:
from sklearn.metrics import mean_squared_error
predictions = model.predict(X_test_noisy)

print(predictions[:20])

print(mean_squared_error(y_test, predictions[:,4]))

NameError: name 'model' is not defined

In [1]:
from qrnn import get_model, qloss
from keras.callbacks import *
import numpy as np

# Generate the synthetic data
x1 = np.sin(np.arange(0, 9, 0.01))
x2 = np.cos(np.arange(0, 9, 0.01))
x3 = x1**2
x4 = (x1+x2)/2

Xtrain = np.vstack((x2, x3, x4)).T #(900, 3)
Ytrain = np.array([x1]*1).T #(900, 1)

# Parameters
input_dim = 3
num_hidden_layers = 2
num_units = [200, 200]
act = ['relu', 'relu']
dropout = [0.1, 0.1]
gauss_std = [0.3, 0.3]
num_quantiles = 9

# Get model
model = get_model(input_dim, num_units, act, dropout, gauss_std, num_hidden_layers, num_quantiles)
print(model.summary())

# Train
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(loss=lambda y_t, y_p: qloss(y_true=y_t, y_pred=y_p, n_q=num_quantiles), 
              optimizer='adam')
model.fit(x=Xtrain, y=Ytrain, 
          epochs=40, 
          validation_split=0.2, 
          batch_size=64, 
          shuffle=True, 
          callbacks=[early_stopping]
         )

2024-03-12 15:20:43.626504: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-12 15:20:43.660737: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 15:20:43.660755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 15:20:43.661598: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-12 15:20:43.667092: I tensorflow/core/platform/cpu_feature_guar

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 3)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │           800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise (GaussianNoise)  │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │        40,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gaussian_noise_1                │ (None, 200)            │             0 │
│ (GaussianNoise)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │         1,809 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,809 (167.22 KB)

 Trainable params: 42,809 (167.22 KB)

...

None
Epoch 1/40


2024-03-12 15:20:46.241701: I external/local_xla/xla/service/service.cc:168] XLA service 0x56398d20bf70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-12 15:20:46.241731: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-12 15:20:46.264778: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-12 15:20:46.507645: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


 1/12 ━━━━━━━━━━━━━━━━━━━━ 49s 5s/step - loss: 1.4174

I0000 00:00:1710256850.119888 1232012 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710256850.133276 1232012 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 388ms/step - loss: 1.2425 - val_loss: 1.0812
Epoch 2/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.0495 - val_loss: 0.9013
Epoch 3/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9771 - val_loss: 0.8178
Epoch 4/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9231 - val_loss: 0.7771
Epoch 5/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.8732 - val_loss: 0.7386
Epoch 6/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8414 - val_loss: 0.7148
Epoch 7/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8004 - val_loss: 0.6772
Epoch 8/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7741 - val_loss: 0.6523
Epoch 9/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7406 - val_loss: 0.6276
Epoch 10/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7153 - val_loss: 0.6076
Epoch 11/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6915 - val_loss: 0.5881
Epoch 12/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6634 - val_loss: 0.553